In [218]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os
from dotenv import load_dotenv
import gspread
import numpy as np
import string
import keyboard

In [239]:
def convertion_to_table(mans_womens, interes_and_category):
    text = mans_womens + interes_and_category
    dirty_data = pd.DataFrame({text.split('\n')[0]:text.split('\n')[1:]}) #Создаем датафрейм из поступающего текста

    dirty_data = pd.concat([dirty_data.loc[1:16], dirty_data.loc[25:44], dirty_data.loc[46:65]]) #Удаляем ненужные строки (они фиксированы)

    #Разделяем таблицу на проценты и категории
    percent = dirty_data[dirty_data[dirty_data.columns[0]].str.contains('%')]
    percent = percent.rename(columns={percent.columns[0]: percent.columns[0][13:-1]})
    
    category = dirty_data[~dirty_data[dirty_data.columns[0]].str.contains('%')]
    category = category.rename(columns={category.columns[0]: ''})
    
    #Создаем очищенную таблицу
    clear_data = percent.set_index(category[''])
    clear_data = clear_data.apply(lambda col: col.map(lambda x: re.sub(r'\s+', ' ', x).replace(' ', '')))
    
    return clear_data.T.reset_index(names='')


def read_login_password(login='login', password='password'):
    load_dotenv()
    return os.getenv(login), os.getenv(password)

# Работа с сайтом

In [220]:
driver = webdriver.Chrome()

The chromedriver version (126.0.6478.63) detected in PATH at C:\ProgramData\chocolatey\bin\chromedriver.exe might not be compatible with the detected chrome version (127.0.6533.119); currently, chromedriver 127.0.6533.119 is recommended for chrome 127.*, so it is advised to delete the driver in PATH and retry


In [221]:
driver.get("https://audience.yandex.ru")
button = driver.find_element(By.CLASS_NAME, 'button2__text') #Поиск кнопки ВОЙТИ
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button).click(button).perform() #Нажатие на кнопку ВОЙТИ

In [222]:
login, password = read_login_password('login','password') #Читаем логин и пароль из скрытого файла

login_element = driver.find_element(By.CSS_SELECTOR, "input[placeholder='Логин или email']") # Находим поле ввода Логина

"""Через команду clear() не удалось очистить поле, т.к. присваивается некое значение value
    и счетчик обновить не получается. Пока использую костыль с ctrl+A и Backspace"""
login_element.send_keys(Keys.CONTROL,'A')
login_element.send_keys(Keys.BACK_SPACE)

login_element.send_keys(login)

"""Нажимаем снова на кнопку войти, что бы перейти к вводу пароля"""
button_2 = driver.find_element(By.CSS_SELECTOR, "button[type='submit']") #Поиск кнопки ВОЙТИ
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button_2).click(button_2).perform() #Нажатие на кнопку ВОЙТИ

In [223]:
"""Вводим пароль"""
password_element = driver.find_element(By.CSS_SELECTOR, "input[name='passwd']") # Находим поле ввода Логина

password_element.send_keys(Keys.CONTROL,'A')
password_element.send_keys(Keys.BACK_SPACE)

password_element.send_keys(password)

"""Нажимаем снова на кнопку войти, что бы наконец войти"""
button_3 = driver.find_element(By.CSS_SELECTOR, "button[type='submit']") #Поиск кнопки ВОЙТИ
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button_3).click(button_3).perform() #Нажатие на кнопку ВОЙТИ

In [224]:
"""Автоматическое закрытие всплывающего окна в правом неижнем углу про CRM"""
try:
    CRM_segments_window = driver.find_element(By.CSS_SELECTOR, "div[class='crm-data-announcement i-bem crm-data-announcement_js_inited crm-data-announcement_shown_yes']")
    CRM_window_close_button = driver.find_element(By.CSS_SELECTOR, "span[class='icon__close icon__close_type_light crm-data-announcement__close icon']")
    CRM_window_close_button.click()

except:
    print('CRM Окно не всплывало, продолжай кликать дальше')

In [225]:
"""Ищем список из кнопок со статистиками"""
# СЛЕДИТЬ ЗА ДЛИНОЙ СПИСКА КНОПОК, ЕСЛИ НАДО ЗАПАРСИТЬ > 30 ЗАПИСЕЙ, ТО ВРУЧНУЮ НА САЙТЕ ВНИЗУ НАЖИМАТЬ "Показать еще", 
# ПОКА ДЛИНА СПИСКА НЕ БУДЕТ БОЛЬШЕ ВАШЕГО ЗАПРОСА НА ПАРСИНГ
# А так же закрывать все всплывающие окошки на сайте

statistick_buttons = driver.find_elements(
    By.CSS_SELECTOR, 
    "span[class='icon__bars audience-segment-row__stats-button audience-segment-row__stats-button_type_uploading audience-segment-row__stats-button_hint_no icon']"
    )
print(f'На странице отображено {len(statistick_button)} записей')

На странице отображено 30 записей


In [226]:
"""Цикл, который будет проходиться по каждой кнопке, внутри делать скриншоты и сохранять их"""
def parcing(num_of_first_button=0, num_of_last_button=None, statistick_buttons = statistick_buttons):

    while len(statistick_buttons) <= num_of_last_button:
        show_more_button = driver.find_element(
                                By.CSS_SELECTOR, 
                                "div[class='audience-segments-table__show-more']"
                                )
        show_more_button.click()
        
        statistick_buttons = driver.find_elements(
                                By.CSS_SELECTOR, 
                                "span[class='icon__bars audience-segment-row__stats-button audience-segment-row__stats-button_type_uploading audience-segment-row__stats-button_hint_no icon']"
                                )
        print(f'Сейчас на странице отображены {len(statistick_buttons)} записей')

    print('Теперь начнем парсить')
    
    slicer_statistick_buttons = statistick_buttons[num_of_first_button:num_of_last_button]
    
    for stat_button in slicer_statistick_buttons:
        stat_button.click() #открытие окна со статистикой
        
        window = driver.find_element(
            By.CSS_SELECTOR, 
            "div[class='audience-segment-statistics audience-segment-statistics_state_ready error-message error-message_type_above i-bem audience-segment-statistics_js_inited']"
            ) #Поиск всего открывшегося окна

        ohvat = driver.find_element(
            By.CSS_SELECTOR, 
            "div[class='audience-stats-widget-amount__wrapper']"
            ).text.split("\n")
        ohvat = re.sub(r'\s+', ' ',ohvat[0]).replace(' ','')
        
        element = driver.find_element(By.CSS_SELECTOR, "div[class='audience-stats-widget-amount__wrapper']")
        driver.execute_script("arguments[0].parentNode.removeChild(arguments[0]);", element) #Удаление ненужной статистики (Охват)
        
        window.screenshot(f'screenshots_mans_womens//{window.text.split('\n')[0]}.png') #Сохранение скриншота окна в папку
        mans_womans_stat = window.text #Парсим данные по статистике женщин мужчин с окна
        
        interess_and_category_button = driver.find_element(
            By.ID, 
            "stat-tabs-tab-2"
            ) #Поиск кнопки Интересы и категории
        
        interess_and_category_button.click()
        
        window = driver.find_element(
            By.CSS_SELECTOR, 
            "div[class='audience-segment-statistics audience-segment-statistics_state_ready error-message error-message_type_above i-bem audience-segment-statistics_js_inited']"
                ) #Поиск окна (хз почему старое не подходит, но надо заново найти окно)

        window.screenshot(f'screenshots_interes//{window.text.split('\n')[0]}.png') #Сохранение скриншота окна в папку
        interes_and_category_text = window.text
        
        if stat_button == slicer_statistick_buttons[0]:
            table = convertion_to_table(mans_womans_stat, interes_and_category_text)
            table['Охват']=ohvat
        else:
            merged_table = convertion_to_table(mans_womans_stat, interes_and_category_text)
            merged_table['Охват']=ohvat
            table = table.merge(merged_table, how='outer')
        
        #Закрытие окна после всех действий
        driver.find_elements(
            By.CSS_SELECTOR, 
            "span[class='icon__close icon__close_new_yes audience-segment-statistics__close icon']")[0].click()
    return table

In [240]:
num_of_first_button = int(input('Введите номер записи, с которой хотите начать парсинг: '))
num_of_last_button = int(input('Введите номер записи, на которой хотите остановиться: '))
table = parcing(num_of_first_button, num_of_last_button)

Введите номер записи, с которой хотите начать парсинг:  1
Введите номер записи, на которой хотите остановиться:  10


Теперь начнем парсить


In [214]:
"""Привожу адресс локаций к нижнему регистру без пунктуаций для джойна с полученной с парсинга таблицей"""
coords_table = pd.read_excel('check.xlsx', sheet_name=0)
coords_table['adress'] = coords_table['Адрес'].str.lower().str.translate(str.maketrans('', '', string.punctuation+'»'+'«'))

"""Тоже привожу все к нижнему регистру в спарсеной таблице"""
table['adress'] = table[''].str.lower().str.translate(str.maketrans('', '', string.punctuation+'»'+'«'))
table.drop(columns='', inplace=True)

table.rename(columns={'Возраст':'<18', '18':'18-25', '25':'25-35', '35':'35-45', '45':'45-55', '55':'>55'}, inplace=True)

In [115]:
full_table = coords_table.merge(table, on='adress', how='left', indicator=True)
full_table.to_excel('Таблица_для_тепловых_карт.xlsx', index=False)

In [ ]:
print("Парсинг выполнен успешно.\nНажмите любую клавишу для завершения...")
keyboard.read_event()  # Ожидает нажатие любой клавиши
driver.quit()